POSITIVI IN ITALIA

In [1]:
!wget https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv 

--2021-02-08 08:47:52--  https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40957 (40K) [text/plain]
Saving to: ‘dpc-covid19-ita-andamento-nazionale.csv’

dpc-covid19-ita-and 100%[===================>]  40.00K  --.-KB/s    in 0.003s  

2021-02-08 08:47:52 (13.1 MB/s) - ‘dpc-covid19-ita-andamento-nazionale.csv’ saved [40957/40957]



In [2]:
!pip3 install plotly

In [3]:
import os
import plotly.graph_objects as go

GRAFICO TOTALE CASI

In [4]:
#ITALIA

covid19_italia = open ("dpc-covid19-ita-andamento-nazionale.csv", "r").readlines()

covid19_italia = [i.split(",") for i in covid19_italia]

data = [i[0] for i in covid19_italia]
totale_casi = [i[13] for i in covid19_italia]

print ("TOTALE CASI:", totale_casi[-1])

print ("TOTALE CASI OGGI: ", data[-1][:10], ": ", int(totale_casi[-1]) - int(totale_casi[-2]))

fig = go.Figure(data=[
    go.Bar(name='data', x=data, y=totale_casi)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

TOTALE CASI: 2636738
TOTALE CASI OGGI:  2021-02-07 :  11640


GRAFICO TOTALE CASI ODIERNI

In [5]:
x = totale_casi[1:]

totale_casi_odierni = [int(x[n])-int(x[n-1]) for n in range(1,len(x))]

fig = go.Figure(data=[
    go.Bar(name='data', x=data, y=totale_casi_odierni)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [6]:
!pip3 install pystan
!pip3 install fbprophet

In [7]:
import pandas as pd
import fbprophet
from fbprophet import Prophet

FB PROPHET PREDICTION

In [8]:
starting_data = "2020-10-01"
predicted_days = 7

for i in data:
    if starting_data in i:
        observation_number = data.index(i)
        observation_number = len(data) - observation_number
        
variable_0 = "data"
variable_1 = data[-(observation_number):]
variable_2 = "totale_casi_odierni"
variable_3 = totale_casi_odierni[-(observation_number):]

df = pd.DataFrame(
    {variable_0: variable_1,
     variable_2: variable_3
    })

df.columns = ["ds","y"]

model = Prophet(yearly_seasonality=False,weekly_seasonality=False, daily_seasonality=False)
model.fit(df)

future = model.make_future_dataframe(periods=predicted_days) #, freq='D'

pred = model.predict(future)

pred[-(predicted_days):]

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
130,2021-02-08 17:00:00,9258.678759,1988.836847,16807.661459,9258.678759,9258.678759,0.0,0.0,0.0,0.0,0.0,0.0,9258.678759
131,2021-02-09 17:00:00,9046.527465,1672.362902,16743.334242,9037.149660,9054.260245,0.0,0.0,0.0,0.0,0.0,0.0,9046.527465
132,2021-02-10 17:00:00,8834.376172,731.068923,16340.121604,8804.078513,8861.495956,0.0,0.0,0.0,0.0,0.0,0.0,8834.376172
133,2021-02-11 17:00:00,8622.224878,732.962882,15759.495404,8570.386426,8673.527837,0.0,0.0,0.0,0.0,0.0,0.0,8622.224878
134,2021-02-12 17:00:00,8410.073584,1008.805162,16185.967101,8322.923510,8491.303806,0.0,0.0,0.0,0.0,0.0,0.0,8410.073584
135,2021-02-13 17:00:00,8197.922291,247.702738,15495.831038,8071.721765,8307.273832,0.0,0.0,0.0,0.0,0.0,0.0,8197.922291
136,2021-02-14 17:00:00,7985.770997,853.513971,15742.293566,7817.045383,8129.311536,0.0,0.0,0.0,0.0,0.0,0.0,7985.770997


In [9]:
date_pred = list(pred['ds'])
date_pred = (date_pred[-(predicted_days):])

yhat = list(pred['yhat'])
yhat = (yhat[-(predicted_days):])
yhat = [round(i,2) for i in yhat]

yhat_lower = list(pred['yhat_lower'])
yhat_lower = (yhat_lower[-(predicted_days):])
yhat_lower = [round(i,2) for i in yhat_lower]

yhat_upper = list(pred['yhat_upper'])
yhat_upper = (yhat_upper[-(predicted_days):])
yhat_upper = [round(i,2) for i in yhat_upper]

date_yhat = zip(date_pred, yhat_lower, yhat, yhat_upper)
date_yhat = (str(list(date_yhat))).replace (",","\n")
print (date_yhat)

fig = go.Figure(data=[
    go.Bar(name='yhat_lower', x=date_pred, y=yhat_lower),
    go.Bar(name='yhat', x=date_pred, y=yhat),
    go.Bar(name='yhat_upper', x=date_pred, y=yhat_upper)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

[(Timestamp('2021-02-08 17:00:00')
 1988.84
 9258.68
 16807.66)
 (Timestamp('2021-02-09 17:00:00')
 1672.36
 9046.53
 16743.33)
 (Timestamp('2021-02-10 17:00:00')
 731.07
 8834.38
 16340.12)
 (Timestamp('2021-02-11 17:00:00')
 732.96
 8622.22
 15759.5)
 (Timestamp('2021-02-12 17:00:00')
 1008.81
 8410.07
 16185.97)
 (Timestamp('2021-02-13 17:00:00')
 247.7
 8197.92
 15495.83)
 (Timestamp('2021-02-14 17:00:00')
 853.51
 7985.77
 15742.29)]
